<a href="https://colab.research.google.com/github/garfield-gray/DataMining/blob/main/Frequent_Pattern_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print('1')

1
